# Preparation

In [1]:
!pip install torch
!pip install torchvision

In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets
from torch.utils.data.dataset import Subset
import matplotlib.pyplot as plt
import random
import os
import enum
import numpy as np
import pandas as pd

C:\Users\karub\Anaconda3\envs\pytorch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\karub\Anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\karub\Anaconda3\envs\pytorch\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Create dataset

In [3]:
class QuestionType(enum.Enum):
    WHICH_ONE = 'which_one'
    IS_IN = 'is_in'
class DataSet(enum.Enum):
    MNIST = 'mnist'
    CIFAR10 = 'cifar10'
    CIFAR100 = 'cifar100'
    FASHION = 'fashion'
    KMNIST = 'kmnist'

get_question_type = {QuestionType.WHICH_ONE: 'which_one',
                       QuestionType.IS_IN: 'is_in'}
get_dataset_name = {DataSet.CIFAR10: 'cifar10',
                    DataSet.MNIST: 'mnist',
                    DataSet.FASHION: 'fashion', 
                    DataSet.KMNIST: 'kmnist'}

In [4]:
get_dataset_name[DataSet.MNIST]

'mnist'

In [5]:
def prepare_mnist_data(batch_size):
    ordinary_train_dataset = datasets.MNIST(root='./data/mnist', train=True, transform=T.ToTensor(), download=True)
    test_dataset = datasets.MNIST(root='./data/mnist', train=False, transform=T.ToTensor())
    train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    full_train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=len(ordinary_train_dataset.data), shuffle=True, num_workers=0)
    num_classes = len(ordinary_train_dataset.classes)
    return full_train_loader, train_loader, test_loader

def prepare_kmnist_data(batch_size):
    ordinary_train_dataset = datasets.KMNIST(root='./data/KMNIST', train=True, transform=T.ToTensor(), download=True)
    test_dataset = datasets.KMNIST(root='./data/KMNIST', train=False, transform=T.ToTensor())
    train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    full_train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=len(ordinary_train_dataset.data), shuffle=True, num_workers=0)
    num_classes = len(ordinary_train_dataset.classes)
    return full_train_loader, train_loader, test_loader

def prepare_fashion_data(batch_size):
    ordinary_train_dataset = datasets.FashionMNIST(root='./data/FashionMnist', train=True, transform=T.ToTensor(), download=True)
    test_dataset = datasets.FashionMNIST(root='./data/FashionMnist', train=False, transform=T.ToTensor())
    train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    full_train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=len(ordinary_train_dataset.data), shuffle=True, num_workers=0)
    num_classes = len(ordinary_train_dataset.classes)
    return full_train_loader, train_loader, test_loader

def prepare_cifar10_data(batch_size):
    train_transform = T.Compose(
        [T.ToTensor(), # transforms.RandomHorizontalFlip(), transforms.RandomCrop(32,4),
         T.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
    test_transform = T.Compose(
        [T.ToTensor(),
         T.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
    ordinary_train_dataset = datasets.CIFAR10(root='./data', train=True, transform=train_transform, download=True)
    test_dataset = datasets.CIFAR10(root='./data', train=False, transform=test_transform)
    train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    full_train_loader = torch.utils.data.DataLoader(dataset=ordinary_train_dataset, batch_size=len(ordinary_train_dataset.data), shuffle=True, num_workers=0)
    return full_train_loader, train_loader, test_loader

full_train_loader_cifar10, train_loader_cifar10, test_loader_cifar10 = prepare_cifar10_data(256)
datas_cifar10, labels_cifar10 = next(iter(full_train_loader_cifar10))
full_train_loader_fashion, train_loader_fashion, test_loader_fashion = prepare_fashion_data(256)
datas_fashion, labels_fashion = next(iter(full_train_loader_fashion))
full_train_loader_kmnist, train_loader_kmnist, test_loader_kmnist = prepare_kmnist_data(256)
datas_kmnist, labels_kmnist = next(iter(full_train_loader_kmnist))
full_train_loader_mnist, train_loader_mnist, test_loader_mnist = prepare_mnist_data(256)
datas_mnist, labels_mnist = next(iter(full_train_loader_mnist))

Files already downloaded and verified


In [6]:
def get_random_dataloader(full_train_loader, batch_size):
    datas, labels = next(iter(full_train_loader))
    count_list, delete_index_list = [0 for i in range(10)], []
    index_list = list(range(len(datas)))
    random.shuffle(index_list)
    for index in index_list:
        target = labels[index].tolist()
        if count_list[target] >= 1000:
            delete_index_list.append(index)
        else:
            count_list[target] += 1
    datas = np.delete(datas, delete_index_list, 0)
    labels = np.delete(labels, delete_index_list, 0)
    
    random_train_matrix_dataset = torch.utils.data.TensorDataset(datas, labels.float())
    full_random_train_loader = torch.utils.data.DataLoader(dataset=random_train_matrix_dataset, batch_size=labels.shape[0], shuffle=False, num_workers=0)
    random_train_loader = torch.utils.data.DataLoader(dataset=random_train_matrix_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    return full_random_train_loader, random_train_loader
full_random_train_loader_cifar10, random_train_loader_cifar10 = get_random_dataloader(full_train_loader_cifar10, 64)

In [7]:
random_train_loader_cifar10

In [8]:
def generate_Q_A_label(single_class_assigned: int, 
                         question_type: int, 
                         all_class_size: int, 
                         question_class_size: int) -> list:
    
    all_class_set = range(all_class_size)
    question_class_set = random.sample(all_class_set, question_class_size)
    temporary_label_list = [0 for i in range(all_class_size)]
    
    if question_type == QuestionType.WHICH_ONE:
        if single_class_assigned in question_class_set:
            for i in [single_class_assigned]:
                temporary_label_list[i] = 1
        
        else:
            comp_question_class_set = [i for i in all_class_set if i not in question_class_set]
            for i in comp_question_class_set:
                temporary_label_list[i] = 1
        
    elif question_type == QuestionType.IS_IN:
        if single_class_assigned in question_class_set:
            for i in question_class_set:
                temporary_label_list[i] = 1
        
        else:
            comp_question_class_set = [i for i in all_class_set if i not in question_class_set]
            for i in comp_question_class_set:
                temporary_label_list[i] = 1
    
    return temporary_label_list

temporary_label_list = generate_Q_A_label(single_class_assigned=0, 
                                            question_type=QuestionType.WHICH_ONE, 
                                            all_class_size=10, 
                                            question_class_size=9)

temporary_label_list

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
def generate_Q_A_labels(labels_tensor: torch.tensor, 
                          question_type: QuestionType,
                          all_class_size: int,
                          question_class_size: int) -> torch.tensor:
    
    labels_list = [int(i) for i in labels_tensor.tolist()]
    Q_A_labels_list = []
    
    for label in labels_list:
        Q_A_label = generate_Q_A_label(single_class_assigned=label, 
                                          question_type=question_type,  # change out of this function
                                          all_class_size=all_class_size, # change out of this function
                                          question_class_size=question_class_size) # change out of this function
        Q_A_labels_list.append(Q_A_label)
        
    Q_A_labels_tensor = torch.tensor(Q_A_labels_list)
    
    return Q_A_labels_tensor

In [11]:
def generate_dataloader_with_Q_A_label(full_random_train_loader: DataLoader, 
                                       question_type: QuestionType,
                                       question_size: int,
                                       batch_size: int) ->[DataLoader, int, int]:
    
    for i, (datas, labels) in enumerate(full_random_train_loader):
        all_class_size = torch.max(labels) + 1 # K is number of classes, full_train_loader is full batch
        all_class_size = int(all_class_size.tolist())
        
    Q_A_labels_tensor = generate_Q_A_labels(labels, question_type, all_class_size, question_size)
    Q_A_labels_matrix_dataset = torch.utils.data.TensorDataset(datas, Q_A_labels_tensor.float())
    
    Q_A_labels_matrix_train_loader = torch.utils.data.DataLoader(dataset=Q_A_labels_matrix_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    full_Q_A_labels_matrix_train_loader = torch.utils.data.DataLoader(dataset=Q_A_labels_matrix_dataset, batch_size=datas.shape[0], shuffle=False, num_workers=0)
    dimension = int(datas.reshape(-1).shape[0]/datas.shape[0])
    return full_Q_A_labels_matrix_train_loader, Q_A_labels_matrix_train_loader, dimension, all_class_size

In [12]:
from traitlets.traitlets import Integer
# Unclear!!!
DataSet_2_DataLoader_generator = {
    DataSet.MNIST: prepare_mnist_data,
    DataSet.CIFAR10: prepare_cifar10_data,
    DataSet.FASHION: prepare_fashion_data,
    DataSet.KMNIST: prepare_kmnist_data
}

def preparence(dataset: DataSet,
                 batch_size: int):
    dataloader_generator = DataSet_2_DataLoader_generator[dataset]
    full_train_loader, train_loader, test_loader = dataloader_generator(batch_size)
    full_random_train_loader, random_train_loader = get_random_dataloader(full_train_loader, batch_size)
    
    return full_random_train_loader, random_train_loader, test_loader

full_random_train_loader, random_train_loader, test_loader = preparence(DataSet.MNIST, 64)

# Model excution

In [13]:
import torch.nn as nn
import torch
import math
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def mae_loss(outputs, labels):
    sm_outputs = F.softmax(outputs, dim=1)
    loss_fn = nn.L1Loss(reduction='none')
    loss_matrix = loss_fn(sm_outputs, labels.float())
    sample_loss = loss_matrix.sum(dim=-1)
    return sample_loss
    
def mse_loss(outputs, labels):
    sm_outputs = F.softmax(outputs, dim=1)
    loss_fn = nn.MSELoss(reduction='none')
    loss_matrix = loss_fn(sm_outputs, labels.float())
    sample_loss = loss_matrix.sum(dim=-1)
    return sample_loss

def gce_loss(outputs, labels):
    q = 0.7
    sm_outputs = F.softmax(outputs, dim=1)
    pow_outputs = torch.pow(sm_outputs, q)
    sample_loss = (1-(pow_outputs*labels).sum(dim=1))/q # n
    return sample_loss

def phuber_ce_loss(outputs, labels):
    trunc_point = 0.1
    n = labels.shape[0]
    soft_max = nn.Softmax(dim=1)
    sm_outputs = soft_max(outputs)
    final_outputs = sm_outputs * labels
    final_confidence = final_outputs.sum(dim=1)
    ce_index = (final_confidence > trunc_point)
    sample_loss = torch.zeros(n).to(device)

    if ce_index.sum() > 0:
        ce_outputs = outputs[ce_index,:]
        logsm = nn.LogSoftmax(dim=-1)
        logsm_outputs = logsm(ce_outputs)
        final_ce_outputs = logsm_outputs * labels[ce_index,:]
        sample_loss[ce_index] = - final_ce_outputs.sum(dim=-1)

    linear_index = (final_confidence <= trunc_point)

    if linear_index.sum() > 0:
        sample_loss[linear_index] = -math.log(trunc_point) + (-1/trunc_point)*final_confidence[linear_index] + 1

    return sample_loss

def ce_loss(outputs, labels):
    logsm = nn.LogSoftmax(dim=1)
    logsm_outputs = logsm(outputs)
    final_outputs = logsm_outputs * labels
    sample_loss = - final_outputs.sum(dim=1)
    return sample_loss

def W_O_loss(loss_fn_, outputs, labels, device, question_class_size, all_class_size):
    n, k = labels.shape[0], labels.shape[1]
    temp_loss = torch.zeros(n, k).to(device)
    for i in range(k):
        tempY = torch.zeros(n, k).to(device)
        tempY[:, i] = 1.0
        temp_loss[:, i] = loss_fn_(outputs, tempY)
        
    candidate_loss = (temp_loss * labels).sum(dim=1)
    noncandidate_loss = (temp_loss * (1-labels)).sum(dim=1)
    total_loss = candidate_loss - ((all_class_size - question_class_size) * (all_class_size - question_class_size - 1))/(question_class_size * (2*all_class_size - question_class_size - 1.0)) * noncandidate_loss
    average_loss = total_loss.mean()
    return average_loss

def I_I_loss(loss_fn_, outputs, labels, device, question_class_size, all_class_size):
    n, k = labels.shape[0], labels.shape[1]
    temp_loss = torch.zeros(n, k).to(device)
    for i in range(k):
        tempY = torch.zeros(n, k).to(device)
        tempY[:, i] = 1.0
        temp_loss[:, i] = loss_fn_(outputs, tempY)
        
    candidate_loss = (temp_loss * labels).sum(dim=1)
    noncandidate_loss = (temp_loss * (1-labels)).sum(dim=1)
    total_loss = candidate_loss - (2*question_class_size**2 + all_class_size**2 - all_class_size*(2*question_class_size + 1))/(2*question_class_size * (all_class_size - question_class_size)) * noncandidate_loss
    average_loss = total_loss.mean()
    return average_loss


In [14]:
class mlp_model(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(mlp_model, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = x.view(-1, self.num_flat_features(x))
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [15]:
from __future__ import absolute_import

import torch.nn as nn
import math


__all__ = ['resnet']

def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, depth, num_classes=1000):
        super(ResNet, self).__init__()
        # Model type specifies number of layers for CIFAR-10 model
        assert (depth - 2) % 6 == 0, 'depth should be 6n+2'
        n = (depth - 2) // 6

        block = Bottleneck if depth >=44 else BasicBlock

        self.inplanes = 16
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self._make_layer(block, 16, n)
        self.layer2 = self._make_layer(block, 32, n, stride=2)
        self.layer3 = self._make_layer(block, 64, n, stride=2)
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)    # 32x32

        x = self.layer1(x)  # 32x32
        x = self.layer2(x)  # 16x16
        x = self.layer3(x)  # 8x8

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x


def resnet(**kwargs):
    """
    Constructs a ResNet model.
    """
    return ResNet(**kwargs)

In [16]:
def accuracy_check(loader, model, device):
    with torch.no_grad():
        total, num_samples = 0, 0
        for images, labels in loader:
            labels, images = labels.to(device), images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += (predicted == labels).sum().item()
            num_samples += labels.size(0) 
    return total / num_samples

In [17]:
def show_loss(epoch,max_epoch, loss):
    print('TRAIN EPOCH[{:03}/{:03}] LOSS:{:03f}'.format(epoch, max_epoch, loss))
def show_acc(epoch,max_epoch, acc, is_val):
    if is_val:
        print('TEST EPOCH[{:03}/{:03}] ACC:{:03f}%'.format(epoch, max_epoch, acc*100))
    else:
        print('TRAIN EPOCH[{:03}/{:03}] ACC:{:03f}%'.format(epoch, max_epoch, acc*100))

In [18]:
def Train(model, 
          question_type, 
          question_class_size, 
          all_class_size, 
          criterion, 
          optimizer, 
          data_loader, 
          device, 
          epoch, 
          max_epoch):
        
    total_loss_train = 0.0
    model.train()

    for n, (data, label) in enumerate(data_loader):
        optimizer.zero_grad()
        data = data.to(device)
        label = label.to(device)
        output = model(data)
        if question_type == QuestionType.WHICH_ONE:
            loss = W_O_loss(criterion, output, label.float(), device, question_class_size, all_class_size)
        elif question_type == QuestionType.IS_IN:
            loss = I_I_loss(criterion, output, label.float(), device, question_class_size, all_class_size)
        else:
            loss = criterion(output,label)

        loss.backward()
        optimizer.step()
        total_loss_train += loss.item()

    show_loss(epoch+1, max_epoch, total_loss_train/(n+1))
    print()
        #show_score(epoch+1, max_epoch, n+1, len(data_loader), total_acc_test/(n+1), is_val=True)
        #print()

    return total_loss_train/(n+1), model

In [19]:
def Eval(model, 
        evaluation_data_loader_train, 
        evaluation_data_loader_test, 
        device, 
        epoch, 
        max_epoch):
    
    model.eval()
    total_acc_train = accuracy_check(evaluation_data_loader_train, model, device)
    total_acc_test = accuracy_check(evaluation_data_loader_test, model, device)


    show_acc(epoch+1, max_epoch, total_acc_train, is_val=False)
    print()
    show_acc(epoch+1, max_epoch, total_acc_test, is_val=True)
    print()
        
    return total_acc_train , total_acc_test, model
        

In [20]:
get_model = {mlp_model: 'MLP',
             resnet: 'ResNet'}
get_loss_function = {ce_loss: 'CE_LOSS', 
                     mae_loss: 'MAE_LOSS'}

In [21]:
from tqdm.notebook import tqdm
import collections

In [29]:
def Q_A_label_Train_Eval(dataset, 
                          question_type, 
                          question_class_size, 
                          model_name, 
                          batch_size, 
                          loss_fn, 
                          EPOCHS):
    print(f'START: {question_type}, {question_class_size}')
    
    full_Q_A_labels_matrix_train_loader, Q_A_labels_matrix_train_loader, dimension, all_class_size = generate_dataloader_with_Q_A_label(full_random_train_loader,
                                                                                                                                       question_type,
                                                                                                                                       question_class_size,
                                                                                                                                       batch_size)
    full_Q_A_datas, full_Q_A_labels = next(iter(full_Q_A_labels_matrix_train_loader))
    counter = collections.Counter(torch.sum(full_Q_A_labels, 1).int().tolist())
    print(counter)
    
    
    DEVICE = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    #if dataset == DataSet.CIFAR10:
    #model = model_name(depth=32, num_classes=all_class_size).to(DEVICE)
    #else:
    model = model_name(input_dim=dimension, hidden_dim=500, output_dim=all_class_size).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

    train_loss_list = []
    train_acc_list = []
    test_acc_list = []
    #criterion=nn.CrossEntropyLoss()
    result_df = pd.DataFrame(columns=["dataset", "epoch", "question_type", "question_class_size", "model", "loss_function", "train_loss", "train_acc", "test_acc"])

    for epoch in tqdm(range(EPOCHS)):
        train_loss, model = Train(model=model, 
                                  question_type=question_type,
                                  question_class_size=question_class_size,
                                  all_class_size=all_class_size,
                                  criterion=loss_fn,
                                  optimizer=optimizer, 
                                  data_loader=Q_A_labels_matrix_train_loader,
                                  device=DEVICE, 
                                  epoch=epoch, 
                                  max_epoch=EPOCHS)
        
        train_acc, test_acc, model = Eval(model=model, 
                                        evaluation_data_loader_train=random_train_loader, 
                                        evaluation_data_loader_test=test_loader, 
                                        device=DEVICE, 
                                        epoch=epoch, 
                                        max_epoch=EPOCHS)

        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        #print(f'TRAIN EPOCH[{epoch+1}/{EPOCHS}] LOSS: {np.mean(train_loss_list)} ACC: {np.mean(train_acc_list)}')
        #print(f'TEST EPOCH[{epoch+1}/{EPOCHS}] ACC:{np.mean(test_acc_list)}')

        result_df = result_df.append({'dataset': get_dataset_name[dataset],
                                      'epoch': epoch+1,
                                      'question_type': get_question_type[question_type], 
                                      'question_class_size': question_class_size, 
                                      'model': get_model[model_name],
                                      'loss_function': get_loss_function[loss_fn],
                                      'train_loss': train_loss,
                                      'train_acc': train_acc,
                                      'test_acc': test_acc}, ignore_index=True)
    if question_type == QuestionType.WHICH_ONE:
        if question_class_size != 9: 
            result_df['label_size_cand'] = counter[1]
            result_df['label_size_comp'] = counter[all_class_size - question_class_size]
        else:
            result_df['label_size_cand'] = counter[1]
            result_df['label_size_comp'] = np.nan
    elif question_type == QuestionType.IS_IN:
        if question_class_size != 5: 
            result_df['label_size_cand'] = counter[question_class_size]
            result_df['label_size_comp'] = counter[all_class_size - question_class_size]
        else:
            result_df['label_size_cand'] = counter[question_class_size]
            result_df['label_size_comp'] = np.nan
    return result_df

In [33]:
for dataset_ in tqdm([DataSet.MNIST]):#DataSet.MNIST, DataSet.KMNIST, DataSet.FASHION
    full_random_train_loader, random_train_loader, test_loader = preparence(dataset_, 512)
    #if dataset_ == DataSet.CIFAR10:
    #model_name = resnet
    #else:
    model_name = mlp_model
    for itr in tqdm(range(1,5)):
        for question_type in tqdm([QuestionType.WHICH_ONE, QuestionType.IS_IN]):
            for question_size in tqdm(range(1, 10)):
                if (itr == 1) and (question_type == QuestionType.WHICH_ONE) and (question_size == 1):
                    results_df = Q_A_label_Train_Eval(dataset=dataset_, 
                                                        question_type=question_type, 
                                                        question_class_size=question_size, 
                                                        model_name=model_name, 
                                                        batch_size=512, 
                                                        loss_fn=mae_loss, 
                                                        EPOCHS=300)
                else:
                    result_df = Q_A_label_Train_Eval(dataset=dataset_, 
                                                        question_type=question_type, 
                                                        question_class_size=question_size, 
                                                        model_name=model_name, 
                                                        batch_size=512, 
                                                        loss_fn=mae_loss, 
                                                        EPOCHS=300)
                    results_df = pd.concat([results_df, result_df])
        results_df.to_csv(f'v2_result_Q_A_mnist_mnist.csv', index=None)

START: QuestionType.WHICH_ONE, 1
Counter({9: 8997, 1: 1003})


TRAIN EPOCH[001/300] LOSS:1.548887

TRAIN EPOCH[001/300] ACC:58.120000%

TEST EPOCH[001/300] ACC:59.440000%

TRAIN EPOCH[002/300] LOSS:0.817454

TRAIN EPOCH[002/300] ACC:81.600000%

TEST EPOCH[002/300] ACC:82.630000%

TRAIN EPOCH[003/300] LOSS:0.452481

TRAIN EPOCH[003/300] ACC:85.960000%

TEST EPOCH[003/300] ACC:86.710000%

TRAIN EPOCH[004/300] LOSS:0.287491

TRAIN EPOCH[004/300] ACC:86.990000%

TEST EPOCH[004/300] ACC:87.370000%

TRAIN EPOCH[005/300] LOSS:0.185411

TRAIN EPOCH[005/300] ACC:88.590000%

TEST EPOCH[005/300] ACC:88.650000%

TRAIN EPOCH[006/300] LOSS:0.212962

TRAIN EPOCH[006/300] ACC:88.830000%

TEST EPOCH[006/300] ACC:88.670000%

TRAIN EPOCH[007/300] LOSS:0.168945

TRAIN EPOCH[007/300] ACC:89.590000%

TEST EPOCH[007/300] ACC:89.260000%

TRAIN EPOCH[008/300] LOSS:0.160388

TRAIN EPOCH[008/300] ACC:89.910000%

TEST EPOCH[008/300] ACC:89.780000%

TRAIN EPOCH[009/300] LOSS:0.158511

TRAIN EPOCH[009/300] ACC:90.040000%

TEST EPOCH[009/300] ACC:89.250000%

TRAIN EPOCH[010/300

TRAIN EPOCH[076/300] LOSS:-0.010054

TRAIN EPOCH[076/300] ACC:91.020000%

TEST EPOCH[076/300] ACC:90.490000%

TRAIN EPOCH[077/300] LOSS:-0.002354

TRAIN EPOCH[077/300] ACC:90.670000%

TEST EPOCH[077/300] ACC:89.840000%

TRAIN EPOCH[078/300] LOSS:0.023399

TRAIN EPOCH[078/300] ACC:90.720000%

TEST EPOCH[078/300] ACC:89.970000%

TRAIN EPOCH[079/300] LOSS:0.017591

TRAIN EPOCH[079/300] ACC:90.860000%

TEST EPOCH[079/300] ACC:90.160000%

TRAIN EPOCH[080/300] LOSS:-0.046094

TRAIN EPOCH[080/300] ACC:90.840000%

TEST EPOCH[080/300] ACC:90.060000%

TRAIN EPOCH[081/300] LOSS:-0.007312

TRAIN EPOCH[081/300] ACC:91.120000%

TEST EPOCH[081/300] ACC:90.480000%

TRAIN EPOCH[082/300] LOSS:0.040202

TRAIN EPOCH[082/300] ACC:91.260000%

TEST EPOCH[082/300] ACC:90.620000%

TRAIN EPOCH[083/300] LOSS:0.014080

TRAIN EPOCH[083/300] ACC:91.070000%

TEST EPOCH[083/300] ACC:90.570000%

TRAIN EPOCH[084/300] LOSS:0.006333

TRAIN EPOCH[084/300] ACC:91.120000%

TEST EPOCH[084/300] ACC:90.510000%

TRAIN EPOCH[085

TRAIN EPOCH[151/300] LOSS:-0.038407

TRAIN EPOCH[151/300] ACC:92.130000%

TEST EPOCH[151/300] ACC:91.580000%

TRAIN EPOCH[152/300] LOSS:-0.010811

TRAIN EPOCH[152/300] ACC:92.140000%

TEST EPOCH[152/300] ACC:91.550000%

TRAIN EPOCH[153/300] LOSS:-0.029750

TRAIN EPOCH[153/300] ACC:92.140000%

TEST EPOCH[153/300] ACC:91.550000%

TRAIN EPOCH[154/300] LOSS:-0.043515

TRAIN EPOCH[154/300] ACC:92.080000%

TEST EPOCH[154/300] ACC:91.490000%

TRAIN EPOCH[155/300] LOSS:0.037521

TRAIN EPOCH[155/300] ACC:92.070000%

TEST EPOCH[155/300] ACC:91.490000%

TRAIN EPOCH[156/300] LOSS:0.030652

TRAIN EPOCH[156/300] ACC:92.040000%

TEST EPOCH[156/300] ACC:91.470000%

TRAIN EPOCH[157/300] LOSS:-0.032215

TRAIN EPOCH[157/300] ACC:92.060000%

TEST EPOCH[157/300] ACC:91.400000%

TRAIN EPOCH[158/300] LOSS:0.030690

TRAIN EPOCH[158/300] ACC:92.050000%

TEST EPOCH[158/300] ACC:91.430000%

TRAIN EPOCH[159/300] LOSS:0.023832

TRAIN EPOCH[159/300] ACC:91.990000%

TEST EPOCH[159/300] ACC:91.470000%

TRAIN EPOCH[16

TRAIN EPOCH[226/300] LOSS:-0.013597

TRAIN EPOCH[226/300] ACC:92.260000%

TEST EPOCH[226/300] ACC:91.640000%

TRAIN EPOCH[227/300] LOSS:-0.028209

TRAIN EPOCH[227/300] ACC:92.040000%

TEST EPOCH[227/300] ACC:91.430000%

TRAIN EPOCH[228/300] LOSS:0.020023

TRAIN EPOCH[228/300] ACC:92.000000%

TEST EPOCH[228/300] ACC:91.320000%

TRAIN EPOCH[229/300] LOSS:-0.000678

TRAIN EPOCH[229/300] ACC:92.200000%

TEST EPOCH[229/300] ACC:91.410000%

TRAIN EPOCH[230/300] LOSS:-0.061923

TRAIN EPOCH[230/300] ACC:92.190000%

TEST EPOCH[230/300] ACC:91.580000%

TRAIN EPOCH[231/300] LOSS:0.026861

TRAIN EPOCH[231/300] ACC:92.190000%

TEST EPOCH[231/300] ACC:91.650000%

TRAIN EPOCH[232/300] LOSS:0.006212

TRAIN EPOCH[232/300] ACC:92.210000%

TEST EPOCH[232/300] ACC:91.720000%

TRAIN EPOCH[233/300] LOSS:-0.024791

TRAIN EPOCH[233/300] ACC:92.280000%

TEST EPOCH[233/300] ACC:91.690000%

TRAIN EPOCH[234/300] LOSS:-0.007675

TRAIN EPOCH[234/300] ACC:92.170000%

TEST EPOCH[234/300] ACC:91.560000%

TRAIN EPOCH[2

...

Counter({9: 8979, 1: 1021})


TRAIN EPOCH[001/300] LOSS:1.408386

TRAIN EPOCH[001/300] ACC:67.140000%

TEST EPOCH[001/300] ACC:68.640000%

TRAIN EPOCH[002/300] LOSS:0.642456

TRAIN EPOCH[002/300] ACC:70.220000%

TEST EPOCH[002/300] ACC:71.450000%

TRAIN EPOCH[003/300] LOSS:0.329430

TRAIN EPOCH[003/300] ACC:85.160000%

TEST EPOCH[003/300] ACC:86.180000%

TRAIN EPOCH[004/300] LOSS:0.167267

TRAIN EPOCH[004/300] ACC:87.610000%

TEST EPOCH[004/300] ACC:87.840000%

TRAIN EPOCH[005/300] LOSS:0.093867

TRAIN EPOCH[005/300] ACC:88.500000%

TEST EPOCH[005/300] ACC:88.890000%

TRAIN EPOCH[006/300] LOSS:-0.009609

TRAIN EPOCH[006/300] ACC:89.060000%

TEST EPOCH[006/300] ACC:89.450000%

TRAIN EPOCH[007/300] LOSS:0.000177

TRAIN EPOCH[007/300] ACC:89.450000%

TEST EPOCH[007/300] ACC:89.390000%

TRAIN EPOCH[008/300] LOSS:-0.071461

TRAIN EPOCH[008/300] ACC:89.280000%

TEST EPOCH[008/300] ACC:89.640000%

TRAIN EPOCH[009/300] LOSS:-0.042082

TRAIN EPOCH[009/300] ACC:89.350000%

TEST EPOCH[009/300] ACC:89.560000%

TRAIN EPOCH[010/

TRAIN EPOCH[076/300] LOSS:-0.151302

TRAIN EPOCH[076/300] ACC:90.720000%

TEST EPOCH[076/300] ACC:90.820000%

TRAIN EPOCH[077/300] LOSS:-0.199637

TRAIN EPOCH[077/300] ACC:90.720000%

TEST EPOCH[077/300] ACC:90.830000%

TRAIN EPOCH[078/300] LOSS:-0.157391

TRAIN EPOCH[078/300] ACC:90.590000%

TEST EPOCH[078/300] ACC:90.660000%

TRAIN EPOCH[079/300] LOSS:-0.095452

TRAIN EPOCH[079/300] ACC:90.780000%

TEST EPOCH[079/300] ACC:90.930000%

TRAIN EPOCH[080/300] LOSS:-0.116112

TRAIN EPOCH[080/300] ACC:90.780000%

TEST EPOCH[080/300] ACC:90.880000%

TRAIN EPOCH[081/300] LOSS:-0.122936

TRAIN EPOCH[081/300] ACC:90.730000%

TEST EPOCH[081/300] ACC:90.890000%

TRAIN EPOCH[082/300] LOSS:-0.137613

TRAIN EPOCH[082/300] ACC:90.620000%

TEST EPOCH[082/300] ACC:90.780000%

TRAIN EPOCH[083/300] LOSS:-0.096261

TRAIN EPOCH[083/300] ACC:90.720000%

TEST EPOCH[083/300] ACC:90.930000%

TRAIN EPOCH[084/300] LOSS:-0.178145

TRAIN EPOCH[084/300] ACC:90.670000%

TEST EPOCH[084/300] ACC:90.800000%

TRAIN EPOC

TRAIN EPOCH[151/300] LOSS:-0.124267

TRAIN EPOCH[151/300] ACC:89.910000%

TEST EPOCH[151/300] ACC:89.640000%

TRAIN EPOCH[152/300] LOSS:-0.062955

TRAIN EPOCH[152/300] ACC:90.770000%

TEST EPOCH[152/300] ACC:90.850000%

TRAIN EPOCH[153/300] LOSS:-0.112063

TRAIN EPOCH[153/300] ACC:90.710000%

TEST EPOCH[153/300] ACC:90.170000%

TRAIN EPOCH[154/300] LOSS:-0.101184

TRAIN EPOCH[154/300] ACC:91.280000%

TEST EPOCH[154/300] ACC:90.930000%

TRAIN EPOCH[155/300] LOSS:-0.076578

TRAIN EPOCH[155/300] ACC:90.950000%

TEST EPOCH[155/300] ACC:90.860000%

TRAIN EPOCH[156/300] LOSS:-0.147073

TRAIN EPOCH[156/300] ACC:90.870000%

TEST EPOCH[156/300] ACC:90.390000%

TRAIN EPOCH[157/300] LOSS:-0.109350

TRAIN EPOCH[157/300] ACC:91.080000%

TEST EPOCH[157/300] ACC:90.460000%

TRAIN EPOCH[158/300] LOSS:-0.051440

TRAIN EPOCH[158/300] ACC:91.320000%

TEST EPOCH[158/300] ACC:91.190000%

TRAIN EPOCH[159/300] LOSS:-0.162268

TRAIN EPOCH[159/300] ACC:91.360000%

TEST EPOCH[159/300] ACC:91.270000%

TRAIN EPOC

TRAIN EPOCH[226/300] LOSS:-0.087162

TRAIN EPOCH[226/300] ACC:91.330000%

TEST EPOCH[226/300] ACC:91.530000%

TRAIN EPOCH[227/300] LOSS:-0.135443

TRAIN EPOCH[227/300] ACC:91.230000%

TEST EPOCH[227/300] ACC:91.620000%

TRAIN EPOCH[228/300] LOSS:-0.100949

TRAIN EPOCH[228/300] ACC:91.240000%

TEST EPOCH[228/300] ACC:91.590000%

TRAIN EPOCH[229/300] LOSS:-0.148449

TRAIN EPOCH[229/300] ACC:91.190000%

TEST EPOCH[229/300] ACC:91.400000%

TRAIN EPOCH[230/300] LOSS:-0.162174

TRAIN EPOCH[230/300] ACC:91.260000%

TEST EPOCH[230/300] ACC:91.510000%

TRAIN EPOCH[231/300] LOSS:-0.134613

TRAIN EPOCH[231/300] ACC:91.290000%

TEST EPOCH[231/300] ACC:91.620000%

TRAIN EPOCH[232/300] LOSS:-0.100185

TRAIN EPOCH[232/300] ACC:91.330000%

TEST EPOCH[232/300] ACC:91.460000%

TRAIN EPOCH[233/300] LOSS:-0.162087

TRAIN EPOCH[233/300] ACC:91.250000%

TEST EPOCH[233/300] ACC:91.460000%

TRAIN EPOCH[234/300] LOSS:-0.133629

TRAIN EPOCH[234/300] ACC:89.850000%

TEST EPOCH[234/300] ACC:89.380000%

TRAIN EPOC

In [56]:
def Ord_label_Train_Eval(dataset, 
                          model, 
                          batch_size, 
                          loss_fn, 
                          EPOCHS):
    Q_A_labels_matrix_train_loader, train_loader, test_loader, dimension, all_class_size = DataSet_2_DataLoader_with_Q_A_label(dataset, 
                                                                                                                                   QuestionType.WHICH_ONE, 
                                                                                                                                   5,
                                                                                                                                   batch_size)
    DEVICE= torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
    model = model(input_dim=dimension, hidden_dim=500, output_dim=all_class_size).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    
    train_loss_list = []

    train_acc_list = []
    test_acc_list = []
    #criterion=nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
        train_loss,train_acc = Train_Eval(model=model, 
                                          question_type=None,
                                          question_class_size=None,
                                          all_class_size=None,
                                          criterion=loss_fn,
                                          optimizer=optimizer, 
                                          data_loader=train_loader,
                                          evaluation_data_loader_train=train_loader,
                                          evaluation_data_loader_test=test_loader,
                                          device=DEVICE, 
                                          epoch=epoch, 
                                          max_epoch=EPOCHS)
        

        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)

        print(f'TRAIN EPOCH[{epoch+1}/{EPOCHS}] LOSS: {train_loss_list.mean()} ACC:{train_acc_list.mean()}')
        print(f'TEST EPOCH[{epoch+1}/{EPOCHS}] ACC:{test_acc_list.mean()}')


In [58]:
Ord_label_Train_Eval(dataset=DataSet.CIFAR10, 
                      model=mlp_model, 
                      batch_size=64, 
                      loss_fn=nn.CrossEntropyLoss(), 
                      EPOCHS=300)

Files already downloaded and verified
TRAIN EPOCH[001/300] ITR [0001/0782] LOSS:2.32732 ACC:0.312250
TEST   EPOCH[001/300] ITR [0001/0782] ACC:0.317812


KeyboardInterrupt: ignored